# TCP Congestion Control

In this notebook you will:

* Create resources for this experiment
* Configure your resources
* Access your resources over SSH
* Retrieve files saved on resources
* Visualize the data retrieved
* Delete your resources, once you are finished

## Exercise: Create resources

In this exercise, we will reserve resources on KVM@TACC: two hosts (VMs) on two different network segments, connected by a router (also a VM).



In [ ]:
# enable autoreload magic to pull in updated utils.py 
%load_ext autoreload
%autoreload 2

In [ ]:
import openstack
import chi
import chi.ssh
import os 

chi.use_site("KVM@TACC")
# set the PROJECT_NAME manually if you don't want the default from the Jupyter environment
PROJECT_NAME = os.getenv('OS_PROJECT_NAME')
chi.set("project_name", PROJECT_NAME)

# configure openstacksdk for actions unsupported by python-chi
os_conn = chi.clients.connection()

In [ ]:
# create three networks. One will be used for SSH and API access,
# and the other two will be used for experiments. 
# We need to disable port security on those two experiment networks.

import utils

uname = os.getenv('USER')
public_net = utils.ensure_network(os_conn, network_name="public-net-" + uname)
exp_net_1  = utils.ensure_network(os_conn, network_name="exp-net-1-" + uname)
exp_net_2  = utils.ensure_network(os_conn, network_name="exp-net-2-" + uname)

public_subnet = utils.ensure_subnet(
    os_conn,
    name="public-subnet-" + uname,
    network_id=public_net.get("id"),
    ip_version='4',
    cidr="192.168.10.0/24",
    gateway_ip="192.168.10.1"
)

exp_subnet_1 = utils.ensure_subnet(
    os_conn,
    name="exp-subnet-1-" + uname,
    network_id=exp_net_1.get("id"),
    ip_version='4',
    cidr="10.10.1.0/24",
    enable_dhcp=True,
    gateway_ip=None
)

exp_subnet_2 = utils.ensure_subnet(
    os_conn,
    name="exp-subnet-2-" + uname,
    network_id=exp_net_2.get("id"),
    ip_version='4',
    cidr="10.10.2.0/24",
    enable_dhcp=True,
    gateway_ip=None
)

In [ ]:
netid_1 = chi.network.get_network_id("exp-net-1-" + uname)
netid_2 = chi.network.get_network_id("exp-net-2-" + uname)

In [ ]:
%%bash -s "$PROJECT_NAME" "$netid_1" "$netid_2"
export OS_PROJECT_NAME=$1
export OS_AUTH_URL=https://kvm.tacc.chameleoncloud.org:5000/v3
export OS_REGION_NAME=KVM@TACC
access_token=$(curl -s -H"authorization: token $JUPYTERHUB_API_TOKEN" "$JUPYTERHUB_API_URL/users/$JUPYTERHUB_USER" | jq -r .auth_state.access_token)
export OS_ACCESS_TOKEN="$access_token"

openstack network set --disable-port-security $2
openstack network set --disable-port-security $3

In [ ]:
# Now, create three servers - romeo, juliet, and router

image_uuid = os_conn.image.find_image("CC-Ubuntu20.04").id
flavor_uuid = os_conn.compute.find_flavor("m1.small").id

server_romeo = utils.ensure_server(
    os_conn,
    name="romeo_" + uname,
    image_id=image_uuid,
    flavor_id=flavor_uuid,
    nics=[
        {"net-id": public_net.get("id"), "v4-fixed-ip":"192.168.10.10"},
        {"net-id": netid_1, "v4-fixed-ip":"10.10.1.100"},
    ]
)

server_juliet = utils.ensure_server(
    os_conn,
    name="juliet_" + uname,
    image_id=image_uuid,
    flavor_id=flavor_uuid,
    nics=[
        {"net-id": public_net.get("id"), "v4-fixed-ip":"192.168.10.20"},
        {"net-id": netid_2, "v4-fixed-ip":"10.10.2.100"},
    ]
)

server_router = utils.ensure_server(
    os_conn,
    name="router_" + uname,
    image_id=image_uuid,
    flavor_id=flavor_uuid,
    nics=[
        {"net-id": public_net.get("id"), "v4-fixed-ip":"192.168.10.30"},
        {"net-id": netid_1, "v4-fixed-ip":"10.10.1.10"},
        {"net-id": netid_2, "v4-fixed-ip":"10.10.2.10"},
    ]
)


In [ ]:
romeo_id  = chi.server.get_server('romeo_' + uname).id
juliet_id = chi.server.get_server('juliet_' + uname).id
router_id = chi.server.get_server('router_' + uname).id

In [ ]:
chi.server.wait_for_active(romeo_id)
chi.server.wait_for_active(juliet_id)
chi.server.wait_for_active(router_id)

In [ ]:
# connect them to the Internet on the "public" network (e.g. for software installation)
router = chi.network.create_router('inet-router-' + uname, gw_network_name='public')
chi.network.add_subnet_to_router(router.get("id"), public_subnet.get("id"))

In [ ]:
# prepare SSH access on the three servers
fip_romeo = chi.server.associate_floating_ip(romeo_id)
fip_juliet = chi.server.associate_floating_ip(juliet_id)
fip_router = chi.server.associate_floating_ip(router_id)

Note: The following cell assumes that a security group named "Allow SSH" already exists in your project, and is configured to allow SSH access on port 22.

In [ ]:
[port_id_1, port_id_2, port_id_3 ] = [port['id'] for port in chi.network.list_ports() if port['port_security_enabled'] and port['network_id']==public_net.get("id")]
security_group_id = os_conn.get_security_group("Allow SSH").id

In [ ]:
%%bash -s "$PROJECT_NAME" "$security_group_id" "$port_id_1" "$port_id_2" "$port_id_3" 

export OS_PROJECT_NAME=$1
export OS_AUTH_URL=https://kvm.tacc.chameleoncloud.org:5000/v3
export OS_REGION_NAME=KVM@TACC
access_token=$(curl -s -H"authorization: token $JUPYTERHUB_API_TOKEN"     "$JUPYTERHUB_API_URL/users/$JUPYTERHUB_USER"     | jq -r .auth_state.access_token)
export OS_ACCESS_TOKEN="$access_token"

openstack port set "$3" --security-group "$2"
openstack port set "$4" --security-group "$2"
openstack port set "$5" --security-group "$2"

## Exercise: Configure resources

Next, we need to configure our resources - assign IP addresses to network interfaces, enable forwarding on the router, and install any necessary software.



In [ ]:
# configure the router to forward traffic
remote_router = chi.ssh.Remote(fip_router) 
remote_router.run(f"sudo sysctl -w net.ipv4.ip_forward=1") 
remote_router.run(f"sudo ufw disable") 
remote_router.run(f"sudo apt update; sudo apt -y install net-tools") 


In [ ]:
# configure the romeo host
remote_romeo = chi.ssh.Remote(fip_romeo) 
remote_romeo.run(f"sudo ip route add 10.10.2.0/24 via 10.10.1.10") 
remote_romeo.run(f"echo '10.10.2.100 juliet' | sudo tee -a /etc/hosts > /dev/null") 
remote_romeo.run(f"sudo apt update; sudo apt -y install net-tools") 

In [ ]:
# configure the juliet host
remote_juliet = chi.ssh.Remote(fip_juliet) 
remote_juliet.run(f"sudo ip route add 10.10.1.0/24 via 10.10.2.10") 
remote_juliet.run(f"echo '10.10.1.100 romeo' | sudo tee -a /etc/hosts > /dev/null") 
remote_juliet.run(f"sudo apt update; sudo apt -y install net-tools") 

Also copy your account keys to all of the resources:

In [ ]:
nova=chi.clients.nova()
# iterate over all keypairs in this account
for kp in nova.keypairs.list(): 
    public_key = nova.keypairs.get(kp.name).public_key 
    remote_router.run(f"echo {public_key} >> ~/.ssh/authorized_keys") 
    remote_romeo.run(f"echo {public_key} >> ~/.ssh/authorized_keys") 
    remote_juliet.run(f"echo {public_key} >> ~/.ssh/authorized_keys") 

## Exercise: Log in to resources

Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the output - you will see an SSH command for each of the nodes in your topology.



In [ ]:
# for romeo:
print(f"ssh cc@{fip_romeo}")

In [ ]:
# for juliet:
print(f"ssh cc@{fip_juliet}")

In [ ]:
# for router:
print(f"ssh cc@{fip_router}")

Now, you can open an SSH session on any of the nodes as follows:

* In Jupyter, from the menu bar, use File > New > Terminal to open a new terminal.
* Copy an SSH command from the output above, and paste it into the terminal. 

You can repeat this process (open several terminals) to start a session on each host and the router. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

Now you can continue to perform the TCP congestion control experiment on these host sessions.



## Exercise: Data Visualization

To visualize the results of your experiment, you should first retrieve the data file `sender-ss.csv` from host romeo using SCP:

In [ ]:
!scp -o StrictHostKeyChecking=no -i ~/work/.ssh/id_rsa cc@{fip_romeo}:/home/cc/sender-ss.csv /work/sender-ss.csv

then run that SCP command in a terminal (File > New > Terminal) in the Jupyter environment to transfer the file.

Then, run the following cell to visualize the results of the experiment.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("/work/sender-ss.csv", names=['time', 'sender', 'retx_unacked', 'retx_cum', 'cwnd', 'ssthresh'])

# exclude the "control" flow
s = df.groupby('sender').size()
df_filtered = df[df.groupby("sender")['sender'].transform('size') > 100]

senders = df_filtered.sender.unique()

time_min = df_filtered.time.min()
cwnd_max = 1.1*df_filtered[df_filtered.time - time_min >=2].cwnd.max()
dfs = [df_filtered[df_filtered.sender==senders[i]] for i in range(3)]

fig, axs = plt.subplots(len(senders), sharex=True, figsize=(12,8))
fig.suptitle('CWND over time')
for i in range(len(senders)):
    if i==len(senders)-1:
        axs[i].plot(dfs[i]['time']-time_min, dfs[i]['cwnd'], label="cwnd")
        axs[i].plot(dfs[i]['time']-time_min, dfs[i]['ssthresh'], label="ssthresh")
        axs[i].set_ylim([0,cwnd_max])
        axs[i].set_xlabel("Time (s)");
    else:
        axs[i].plot(dfs[i]['time']-time_min, dfs[i]['cwnd'])
        axs[i].plot(dfs[i]['time']-time_min, dfs[i]['ssthresh'])
        axs[i].set_ylim([0,cwnd_max])


plt.tight_layout();
fig.legend(loc='upper right', ncol=2);

## Exercise: Transfer .pcap files from a host

In the TCP ECN exercise, retrieve the PCAP files from the hosts romeo and juliet using the following commands:


In [ ]:
romeo_pcap = "/home/ubuntu/%s-tcp-ecn.pcap" % remote_romeo.run("hostname")
!scp -o StrictHostKeyChecking=no -i ~/work/.ssh/id_rsa cc@{fip_romeo}:/home/cc/{romeo_pcap} /work/

In [ ]:
juliet_pcap = "/home/ubuntu/%s-tcp-ecn.pcap" % remote_juliet.run("hostname")
!scp -o StrictHostKeyChecking=no -i ~/work/.ssh/id_rsa cc@{fip_juliet}:/home/cc/{juliet_pcap} /work/

Then in the Jupyter environment, click on the folder icon in the file browser on the left to make sure that you are located in your “Jupyter home” directory.

Then, you should see the above .pcap files appear in the Jupyter file browser on the left. You can now download this file from the Jupyter environment to your own laptop to analyze them in Wireshark.



## Delete resources

To free your resources, change the condition in the following cell from `False` to `True`, then run the cell.

In [ ]:
if False:
    
    for server_id in [romeo_id, juliet_id, router_id]:
        chi.server.delete_server(server_id)
        
    for reserved_fip in [fip_romeo, fip_juliet, fip_router]:
        ip_details = chi.network.get_floating_ip(reserved_fip)
        chi.neutron().delete_floatingip(ip_details["id"])
        
    chi.network.remove_subnet_from_router(router.get("id"), public_subnet.get("id"))
    chi.network.delete_router(router.get("id"))
    
    chi.network.delete_subnet(public_subnet.get('id'))
    chi.network.delete_network(public_net.get("id"))
    
    chi.network.delete_subnet(exp_subnet_1.get('id'))
    chi.network.delete_network(exp_net_1.get("id"))

    chi.network.delete_subnet(exp_subnet_2.get('id'))
    chi.network.delete_network(exp_net_2.get("id"))